# Stock Analysis Report Generator Using Perplexity

<small>

#### **Overview**
This notebook automatically generates professional stock analysis reports in the style of Brian Belsky (Chief Investment Strategist at BMO Capital Markets) using Perplexity AI's Sonar Pro model. Each report is saved as a formatted Microsoft Word document.6. The prompt used to generate the report
Analyst ratings and price targets (when available)

#### **Features**
- Batch Processing: Processes multiple stocks from a text file3. Key financial metrics table
- Smart Skipping: Avoids duplicate API calls by checking for existing reports2. 5-year price performance vs. NASDAQ
- Markdown Formatting: Converts AI-generated markdown to professional Word formatting1. Stock ticker and generation date
- Comprehensive Analysis: Includes price performance charts, financial metrics, and analyst ratingsEach report includes:
- Date Stamping: Automatically timestamps each report## Output Format

#### **Requirements**
- Perplexity API Key- 
- Input file: Text file with stock tickers (one per line)
- Python 3.7+

#### **Output**
- Analysis word document for each equity




## Import Libraries

In [6]:
# Check which packages are available and install missing ones
# Install from requirements file
!pip install -q -r requirements.txt

print("✅ Package installation examples shown above!")

# Import necessary packages
import sys
import requests
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
import os.path
from os.path import isfile, join
from os import listdir
import subprocess
import json
import sys
from datetime import datetime
from typing import Dict, List, Optional, Any
from IPython.display import Markdown, display
from IPython.display import SVG
import numpy as np
from time import time
np.random.seed(10)
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import argparse
import random
import sys

✅ Package installation examples shown above!


## Configuration Settings

In [7]:
### Variable detail
# **MAX_TOKENS**: `2000` - Maximum length of generated response
# **INPUT_DIR**: Directory containing the equity list file- **TEMPERATURE**: `0` - Deterministic output for consistent, factual analysis
# **OUTPUT_DIR**: Directory where generated reports will be saved- **MODEL**: `sonar-pro` - Perplexity's most advanced model with real-time web access and current financial data
# **EQUITY_LIST_FILE**: Text file with stock tickers (one per line)
# **Model Settings: Specific model to be invoked, 0 temperature to eliminate creativity and limit the number of tokens for the query.



# Load environment variables
load_dotenv()

# Get API key
API_KEY = os.getenv("PERPLEXITY_API_KEY")


# Input/Output Configuration
INPUT_DIR = os.getenv("Input_dir")
OUTPUT_DIR = os.getenv("Output_dir")
PROMPT_DIR = os.getenv("Prompt_dir")
EQUITY_LIST_FILE = os.path.join(INPUT_DIR, "Equity_list_test.txt").replace('\\', '/')
PROMPT_FILE = os.path.join(PROMPT_DIR, "Individual_analysis_prompt.txt")

# Read stock list from file
with open(EQUITY_LIST_FILE, 'r') as f:
    EQUITY_LIST = [line.strip() for line in f if line.strip()]

# Read prompt template from file
with open(PROMPT_FILE, 'r') as f:
    PROMPT_TEMPLATE = f.read().strip()
print(PROMPT_TEMPLATE)

# Model Configuration
MODEL = "sonar-pro" 
TEMPERATURE = 0
MAX_TOKENS = 2000


give me a Brian Belsky style stock analysis for this actual stock market stock in a 3 page or less format. 
Insert a chart at the top with trailing 5 year stock price performance of this stock vs. the 
Nasdaq as well as a table of all the key financial metrics for this stock. Make certain all of the facts are correct.  
If available, please add analyst ratings and price targets for this stock or mutual fund.


## Define Class to Call Model API

In [8]:
from openai import OpenAI

class PerplexityClient:
    def __init__(self, api_key):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.perplexity.ai"
        )

    def chat(self, message, model=MODEL):
        response = self.client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}]
        )
        return response.choices[0].message.content

## Define Class to Format the Output in a Document

In [9]:

def add_markdown_to_word(doc, markdown_text):
    """Convert markdown text to formatted Word document content"""
    lines = markdown_text.split('\n')
    i = 0
    
    while i < len(lines):
        line = lines[i]
        
        # Skip empty lines
        if not line.strip():
            i += 1
            continue
        
        # Headers (# ## ###)
        if line.startswith('#'):
            level = len(line) - len(line.lstrip('#'))
            text = line.lstrip('#').strip()
            doc.add_heading(text, level=min(level, 9))
        
        # Tables (|...)
        elif '|' in line and i + 1 < len(lines) and '|' in lines[i + 1]:
            # Parse table
            table_lines = [line]
            i += 1
            # Skip separator line
            if '---' in lines[i] or ':-:' in lines[i]:
                i += 1
            # Get table rows
            while i < len(lines) and '|' in lines[i]:
                table_lines.append(lines[i])
                i += 1
            
            # Create table
            headers = [cell.strip() for cell in table_lines[0].split('|') if cell.strip()]
            num_cols = len(headers)
            num_rows = len(table_lines)
            
            table = doc.add_table(rows=num_rows, cols=num_cols)
            table.style = 'Light Grid Accent 1'
            
            # Add headers
            for j, header in enumerate(headers):
                cell = table.rows[0].cells[j]
                cell.text = header
                cell.paragraphs[0].runs[0].bold = True
            
            # Add data rows
            for row_idx in range(1, len(table_lines)):
                cells = [cell.strip() for cell in table_lines[row_idx].split('|') if cell.strip()]
                for col_idx, cell_text in enumerate(cells):
                    if col_idx < num_cols:
                        table.rows[row_idx].cells[col_idx].text = cell_text
            
            doc.add_paragraph()  # Add space after table
            continue
        
        # Bullet points (- or *)
        elif line.strip().startswith(('- ', '* ', '• ')):
            text = line.strip()[2:].strip()
            # Handle bold/italic in bullet points
            p = doc.add_paragraph(style='List Bullet')
            add_formatted_text(p, text)
        
        # Numbered lists (1. 2. etc)
        elif re.match(r'^\d+\.\s', line.strip()):
            text = re.sub(r'^\d+\.\s', '', line.strip())
            p = doc.add_paragraph(style='List Number')
            add_formatted_text(p, text)
        
        # Bold text with **
        elif '**' in line or '__' in line:
            p = doc.add_paragraph()
            add_formatted_text(p, line)
        
        # Regular paragraph
        else:
            p = doc.add_paragraph()
            add_formatted_text(p, line)
        
        i += 1

def add_formatted_text(paragraph, text):
    """Add text with bold/italic markdown formatting to a paragraph"""
    # Handle **bold** and __bold__
    parts = re.split(r'(\*\*.*?\*\*|__.*?__|`.*?`)', text)
    
    for part in parts:
        if not part:
            continue
        
        if part.startswith('**') and part.endswith('**'):
            run = paragraph.add_run(part[2:-2])
            run.bold = True
        elif part.startswith('__') and part.endswith('__'):
            run = paragraph.add_run(part[2:-2])
            run.bold = True
        elif part.startswith('`') and part.endswith('`'):
            run = paragraph.add_run(part[1:-1])
            run.font.name = 'Courier New'
        else:
            paragraph.add_run(part)

## Generate Analysis for Each Stock in a List

In [ ]:
from datetime import datetime
import os

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize Perplexity client
client = PerplexityClient(api_key=API_KEY)
counter = 0
# Process each equity
for equity in EQUITY_LIST:
 
    # Construct prompt with equity ticker and template from file
    prompt = f"For the equity {equity} {PROMPT_TEMPLATE}"
    print(f"Processing: {equity}")
    
    # Check if report already exists for today
    date_str = datetime.now().strftime('%Y-%m-%d')
    output_filename = f"Equity Report - {equity} {date_str}.docx"
    output_path = os.path.join(OUTPUT_DIR, output_filename)
    
    if os.path.exists(output_path):
        print(f"⏭️  Skipping {equity} - report already exists for {date_str}")
        continue

    try:
        # Query the LLM
        print(f"Querying Perplexity API for {equity}...")
        
        generated_text = client.chat(prompt, model=MODEL)
        counter += 1
        print(counter)
        
        # Create Word document
        doc = Document()
        doc.add_heading(f"Market Outlook Report - {equity}", 0)
        doc.add_paragraph(f"Perplexity Sonar Pro Model Generated: {date_str}")
        doc.add_paragraph()
        
        # Convert markdown to Word formatting
        add_markdown_to_word(doc, generated_text)
        
        # Add prompt at the end
        doc.add_paragraph()
        doc.add_heading("Prompt Used:", level=2)
        doc.add_paragraph(prompt)
        
        # Save the document
        doc.save(output_path)
        print(f"✅ Saved: {output_filename}")
        
    except Exception as e:
        print(f"❌ Error processing {equity}: {e}")
        continue
print(counter) 
print(f"\n{'='*60}")
print(f"✅ Completed processing {len(EQUITY_LIST)} equities")
print(f"Reports saved to: {OUTPUT_DIR}")
print(f"{'='*60}")

Processing: PSLV
Querying Perplexity API for PSLV...
